## Feature extractor selection

### Settings

Import the libraries

In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

c:\Users\markz\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


2.10.0


In [2]:
# Random seed for reproducibility
seed = 45

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [3]:
# Labels
LABELS = ['Species1','Species2','Species3','Species4','Species5','Species6','Species7','Species8']

In [4]:
# Model hyperparameters
INPUT_SHAPE = (96, 96, 3)
IMAGE_SIZE = (INPUT_SHAPE[0], INPUT_SHAPE[1])

EPOCHS = 100
BATCH_SIZE = 64
LEARNING_RATE = 1e-3

NUM_CLASSES = len(LABELS)
MODEL_NAME = "cnn"

In [5]:
# Paths
DATA_DIR = 'C:\\Users\\markz\\Desktop\\ANN_chall_1\\training_data_final'

PATH = {}
PATH['training'] = os.path.join(DATA_DIR, 'training')
PATH['validation'] = os.path.join(DATA_DIR, 'validation')
PATH['testing'] = os.path.join(DATA_DIR, 'testing')

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constructor
train_data_gen = ImageDataGenerator(
    rotation_range=30,
    height_shift_range=40,
    width_shift_range=40,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.7,1.3],
    fill_mode='reflect'
) 

# Generator
train_gen = train_data_gen.flow_from_directory(
    directory=PATH['training'],
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=seed
)

Found 2122 images belonging to 8 classes.


In [7]:
unique, counts = np.unique(train_gen.classes, return_counts=True)
rec = dict(zip(unique, counts))

max_value = rec[max(rec, key=rec.get)]

cw = {}

for k in rec.keys():
    cw[k] = max_value/rec[k]

#print(cw)

cw[0] = 5.0
for i in range(1,8):
    cw[i] = 1.0
cw[5] = 3.0
cw[7] = 1.3

print(cw)

{0: 5.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 3.0, 6: 1.0, 7: 1.3}


In [8]:
# Constructor
valid_data_gen = ImageDataGenerator()

# Generator
valid_gen = valid_data_gen.flow_from_directory(
    directory=PATH['validation'],
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=seed
)

Found 707 images belonging to 8 classes.


In [9]:
# Constructor
test_data_gen = ImageDataGenerator()

# Generator
test_gen = test_data_gen.flow_from_directory(
    directory=PATH['testing'],
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    classes=None,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False,
    seed=seed
)

Found 713 images belonging to 8 classes.


In [10]:
LAYERS = [[512],
          [256],
          [128],
          [128,128],
          [256,128],
          [512,128],
          [512,256,128],
          [128,256,128]]
FREEZE = [0, round(0.3*295), round(0.6*295), round(0.9*295)]

In [11]:
# Use the supernet as feature extractor

def train(layers,layers_to_freeze):
    pretrained_model= tf.keras.applications.ConvNeXtBase(include_top=False,input_shape=INPUT_SHAPE,weights='imagenet')

    MODEL_NAME = 'convnext_base'

    pretrained_model.trainable = False

    # Rebuild the classifier
    inputs = tfk.Input(shape=INPUT_SHAPE)
    x = pretrained_model(inputs)
    x = tfkl.GlobalAveragePooling2D()(x)

    for layer in layers:
        x = tfkl.Dense(layer, kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)
        x = tfkl.Dropout(0.3)(x)
        x = tfkl.BatchNormalization()(x)
        x = tfkl.ReLU()(x)

    outputs = tfkl.Dense(
        NUM_CLASSES,
        activation='softmax',
        kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

    model.get_layer(MODEL_NAME).trainable = True

    for i, layer in enumerate(model.get_layer(MODEL_NAME).layers[:layers_to_freeze]):
        layer.trainable=False

    model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(learning_rate=LEARNING_RATE), metrics=['accuracy'])

    print('Fitting with layers',layers,'and freeze '+str(layers_to_freeze))

    with tf.device("/GPU:0"):
        history = model.fit(
            x = train_gen,
            batch_size = BATCH_SIZE,
            epochs = EPOCHS,
            validation_data = valid_gen,
            class_weight=cw,
            verbose=0,
            callbacks=[tfk.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', patience=15, restore_best_weights=True),tfk.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, min_lr=0.0, min_delta=0.00001)]
        ).history

    print('Accuracy: '+str(max(history['val_accuracy'])))
    
    return (layers, layers_to_freeze, max(history['val_accuracy']))


In [12]:
results = []
for l in LAYERS:
    for f in FREEZE:
        results.append(train(l,f))

Fitting with layers [512] and freeze 0
Accuracy: 0.9009901285171509
Fitting with layers [512] and freeze 88
Accuracy: 0.9193776249885559
Fitting with layers [512] and freeze 177
Accuracy: 0.8882602453231812
Fitting with layers [512] and freeze 266
Accuracy: 0.8698726892471313
Fitting with layers [256] and freeze 0
Accuracy: 0.896746814250946
Fitting with layers [256] and freeze 88
Accuracy: 0.9236209392547607
Fitting with layers [256] and freeze 177
Accuracy: 0.8981612324714661
Fitting with layers [256] and freeze 266
Accuracy: 0.8712871074676514
Fitting with layers [128] and freeze 0
Accuracy: 0.893917977809906
Fitting with layers [128] and freeze 88
Accuracy: 0.9278641939163208
Fitting with layers [128] and freeze 177
Accuracy: 0.8896746635437012
Fitting with layers [128] and freeze 266
Accuracy: 0.8727015852928162
Fitting with layers [128, 128] and freeze 0
Accuracy: 0.8783592581748962
Fitting with layers [128, 128] and freeze 88
Accuracy: 0.9193776249885559
Fitting with layers [128

In [19]:
for r in results: print(r)

([512], 0, 0.9009901285171509)
([512], 88, 0.9193776249885559)
([512], 177, 0.8882602453231812)
([512], 266, 0.8698726892471313)
([256], 0, 0.896746814250946)
([256], 88, 0.9236209392547607)
([256], 177, 0.8981612324714661)
([256], 266, 0.8712871074676514)
([128], 0, 0.893917977809906)
([128], 88, 0.9278641939163208)
([128], 177, 0.8896746635437012)
([128], 266, 0.8727015852928162)
([128, 128], 0, 0.8783592581748962)
([128, 128], 88, 0.9193776249885559)
([128, 128], 177, 0.896746814250946)
([128, 128], 266, 0.8599717020988464)
([256, 128], 0, 0.9024045467376709)
([256, 128], 88, 0.9123055338859558)
([256, 128], 177, 0.8854314088821411)
([256, 128], 266, 0.8599717020988464)
([512, 128], 0, 0.8797736763954163)
([512, 128], 88, 0.9250353574752808)
([512, 128], 177, 0.8896746635437012)
([512, 128], 266, 0.8599717020988464)
([512, 256, 128], 0, 0.806223452091217)
([512, 256, 128], 88, 0.908062219619751)
([512, 256, 128], 177, 0.8868458271026611)
([512, 256, 128], 266, 0.8628005385398865)
([

In [13]:
"""plt.figure(figsize=(10,5))
plt.plot(history['loss'], alpha=.3, color='r', linestyle='--', linewidth=3)
plt.plot(history['val_loss'], label=MODEL_NAME, alpha=.8, color='r', linewidth=3)
plt.legend(loc='upper right', prop={'size': 18})
plt.title('Categorical Crossentropy', fontsize=20)
plt.grid(alpha=.3)

plt.figure(figsize=(10,5))
plt.plot(history['accuracy'], alpha=.3, color='r', linestyle='--', linewidth=3)
plt.plot(history['val_accuracy'], label=MODEL_NAME, alpha=.8, color='r', linewidth=3)
plt.legend(loc='upper right', prop={'size': 18})
plt.title('Accuracy', fontsize=20)
plt.grid(alpha=.3)

plt.show()"""

"plt.figure(figsize=(10,5))\nplt.plot(history['loss'], alpha=.3, color='r', linestyle='--', linewidth=3)\nplt.plot(history['val_loss'], label=MODEL_NAME, alpha=.8, color='r', linewidth=3)\nplt.legend(loc='upper right', prop={'size': 18})\nplt.title('Categorical Crossentropy', fontsize=20)\nplt.grid(alpha=.3)\n\nplt.figure(figsize=(10,5))\nplt.plot(history['accuracy'], alpha=.3, color='r', linestyle='--', linewidth=3)\nplt.plot(history['val_accuracy'], label=MODEL_NAME, alpha=.8, color='r', linewidth=3)\nplt.legend(loc='upper right', prop={'size': 18})\nplt.title('Accuracy', fontsize=20)\nplt.grid(alpha=.3)\n\nplt.show()"

In [14]:
"""test_steps_per_epoch = np.math.ceil(test_gen.samples / test_gen.batch_size)

predictions = model.predict(test_gen, steps=test_steps_per_epoch)"""

'test_steps_per_epoch = np.math.ceil(test_gen.samples / test_gen.batch_size)\n\npredictions = model.predict(test_gen, steps=test_steps_per_epoch)'

In [15]:
"""predicted_classes = np.argmax(predictions, axis=-1)
true_classes = test_gen.classes
class_labels = list(test_gen.class_indices.keys())"""

'predicted_classes = np.argmax(predictions, axis=-1)\ntrue_classes = test_gen.classes\nclass_labels = list(test_gen.class_indices.keys())'

In [16]:
"""cm = confusion_matrix(true_classes, predicted_classes)

accuracy = accuracy_score(true_classes, predicted_classes)
precision = precision_score(true_classes, predicted_classes, average='macro')
recall = recall_score(true_classes, predicted_classes, average='macro')
f1 = f1_score(true_classes, predicted_classes, average='macro')
print('Accuracy:',accuracy.round(4))
print('Precision:',precision.round(4))
print('Recall:',recall.round(4))
print('F1:',f1.round(4))

plt.figure(figsize=(10,8))
sns.heatmap(cm.T, xticklabels=list(class_labels), yticklabels=class_labels)
plt.xlabel('True labels')
plt.ylabel('Predicted labels')
plt.show()"""

"cm = confusion_matrix(true_classes, predicted_classes)\n\naccuracy = accuracy_score(true_classes, predicted_classes)\nprecision = precision_score(true_classes, predicted_classes, average='macro')\nrecall = recall_score(true_classes, predicted_classes, average='macro')\nf1 = f1_score(true_classes, predicted_classes, average='macro')\nprint('Accuracy:',accuracy.round(4))\nprint('Precision:',precision.round(4))\nprint('Recall:',recall.round(4))\nprint('F1:',f1.round(4))\n\nplt.figure(figsize=(10,8))\nsns.heatmap(cm.T, xticklabels=list(class_labels), yticklabels=class_labels)\nplt.xlabel('True labels')\nplt.ylabel('Predicted labels')\nplt.show()"

In [17]:
# model.save("cnn_model")